# Transfer Learning

<https://www.kaggle.com/aryan7781/transfer-model-vgg-16>

In [1]:
from pathlib import Path

BASE_DIR = Path('/Users/efraflores/Desktop/hub/datathon2021/Scripts/data/test/Transfer_Learning')
EPOCHS = 4
BATCH_SIZE = 1500

## Import dataset

In [2]:
from tensorflow.keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print(X_train.shape)

(50000, 32, 32, 3)


## Change dim

In [3]:
import cv2
import numpy as np

def gray_to_rgb(X):
    """Expands the array to match the shape of a RGB image"""
    X = map(lambda x: np.stack((x,)*3, axis=-1), X)
    return np.array(list(X))

def min_size(X):
    """Resize the array to match the minimum size of pre-trained model"""
    if X.shape[1] < 32 or X.shape[2] < 32:
        X = np.array([cv2.resize(x, dsize=(32,32), interpolation=cv2.INTER_CUBIC) for x in X])
    return X

In [4]:
# X_train = min_size(gray_to_rgb(X_train))
# X_test = min_size(gray_to_rgb(X_test))
# print(X_train.shape)

In [5]:
from tensorflow.keras.utils import to_categorical

# Reshape the target to be an array of 0 and 1 where the class is present
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(y_train.shape)

(50000, 10)


## Model

### VGG16

In [6]:
import os
from tensorflow.keras.models import Model
from tensorflow.keras.applications import vgg16
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM, GRU
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Get pretrained arquitecture and weights
pretrained_model = vgg16.VGG16(include_top=False, input_shape=X_train.shape[1:], pooling='max')
pretrained_model.trainable = False

# Set the input
inputs = pretrained_model.input

# Add layers to fine-tuning (learn from your data)
x = Dense(128, activation='relu')(pretrained_model.output)
x = Dense(128, activation='relu')(x)

# If there's a categorical task, final layer should have N units where N is the number of classes
outputs = Dense(y_train.shape[-1], activation='softmax')(x)

# Set the model, from inputs to outputs
model = Model(inputs=inputs, outputs=outputs)

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
# Callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20)
checkpoint = ModelCheckpoint(BASE_DIR.joinpath('models','transfer_vgg16_{val_accuracy:.3f}.h5'),
                             save_best_only=True,
                             save_weights_only=False,
                             monitor='val_accuracy')

# There shoul be a train and test generator
training_history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping])

Epoch 1/4
34/34 [==============================] - 190s 6s/step - loss: 5.4746 - accuracy: 0.3180 - val_loss: 2.4861 - val_accuracy: 0.3884
Epoch 2/4
34/34 [==============================] - 223s 7s/step - loss: 2.0670 - accuracy: 0.4222 - val_loss: 1.8567 - val_accuracy: 0.4370
Epoch 3/4
34/34 [==============================] - 262s 8s/step - loss: 1.6531 - accuracy: 0.4771 - val_loss: 1.6583 - val_accuracy: 0.4690
Epoch 4/4
34/34 [==============================] - 267s 8s/step - loss: 1.4673 - accuracy: 0.5152 - val_loss: 1.5500 - val_accuracy: 0.4988


### ResNet50

In [7]:
import os
from tensorflow.keras.models import Model
from tensorflow.keras.applications import resnet50
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM, GRU
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Get pretrained arquitecture and weights
pretrained_model = resnet50.ResNet50(include_top=False, input_shape=X_train.shape[1:], pooling='max')
pretrained_model.trainable = False

# Set the input
inputs = pretrained_model.input

# Add layers to fine-tuning (learn from your data)
x = Dense(128, activation='relu')(pretrained_model.output)
x = Dense(128, activation='relu')(x)

# If there's a categorical task, final layer should have N units where N is the number of classes
outputs = Dense(y_train.shape[-1], activation='softmax')(x)

# Set the model, from inputs to outputs
model = Model(inputs=inputs, outputs=outputs)

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
# Callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20)
checkpoint = ModelCheckpoint(BASE_DIR.joinpath('models','transfer_resnet50_{val_accuracy:.3f}.h5'),
                             save_best_only=True,
                             save_weights_only=False,
                             monitor='val_accuracy')

# There shoul be a train and test generator
training_history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE//2, validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping])

Epoch 1/4
67/67 [==============================] - 100s 1s/step - loss: 1.5370 - accuracy: 0.4724 - val_loss: 1.2285 - val_accuracy: 0.5720


/Users/efraflores/Desktop/hub/datathon2021/venv/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/4
67/67 [==============================] - 119s 2s/step - loss: 1.1244 - accuracy: 0.6046 - val_loss: 1.1440 - val_accuracy: 0.5983
Epoch 3/4
67/67 [==============================] - 95s 1s/step - loss: 1.0086 - accuracy: 0.6463 - val_loss: 1.1037 - val_accuracy: 0.6163
Epoch 4/4
67/67 [==============================] - 117s 2s/step - loss: 0.9248 - accuracy: 0.6737 - val_loss: 1.1006 - val_accuracy: 0.6150
